# **Movie recommendation system**

### Would we be able to predict which movies might or might not be a commercial success? This dataset collects part of the knowledge from the API TMDB, which contains only 5000 movies out of the total number.

In [1]:
# Import the libraries
import os
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

### **Step 1:** Loading the dataset

In [2]:
# Load the dataframe tmdb_5000_movies: https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv
tmdb_5000_movies_df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv')
tmdb_5000_movies_df.to_csv('../data/raw/tmdb_5000_movies.csv', index = False)
tmdb_5000_movies_df

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [3]:
# Load the dataframe tmdb_5000_credits: https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv
tmdb_5000_credits_df = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv')
tmdb_5000_credits_df.to_csv('../data/raw/tmdb_5000_credits.csv', index = False)
tmdb_5000_credits_df.rename(columns={'cast':'casting'}, inplace=True)
tmdb_5000_credits_df

,movie_id,title,casting,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


### **Step 2:** Creation of a database

In [4]:
# Conect to Postgres Database

# load the .env file variables
load_dotenv()

# Connect to the database using the SQLAlchemy's create_engine function
def connect():
    global engine

connection_string = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
print("Starting the connection...")
engine = create_engine(connection_string).execution_options(autocommit=True)
engine.connect()


Starting the connection...


In [5]:
# Create the tables tmdb_5000_movies and 
engine.execute("""
DROP TABLE IF EXISTS tmdb_5000_movies;

DROP TABLE IF EXISTS tmdb_5000_credits;
""")

engine.execute(
    """
    CREATE TABLE IF NOT EXISTS tmdb_5000_movies(
        index INT,
        budget INT,
        genres VARCHAR(10000),
        homepage VARCHAR(10000),
        id INT NOT NULL,
        keywords VARCHAR(10000),
        original_language VARCHAR(10000),
        original_title VARCHAR(10000),
        overview VARCHAR(10000),
        popularity FLOAT,
        production_companies VARCHAR(10000),
        production_countries VARCHAR(10000),
        release_date TIMESTAMP,
        revenue BIGINT,
        runtime FLOAT,
        spoken_languages VARCHAR(10000),
        status VARCHAR(10000),
        tagline VARCHAR(10000),
        title VARCHAR(10000),
        vote_average FLOAT,
        vote_count INT,
        PRIMARY KEY(id)
    );
    
    CREATE TABLE IF NOT EXISTS tmdb_5000_credits(
        index INT NOT NULL,
        movie_id INT,
        title VARCHAR(100000),
        casting VARCHAR(100000),
        crew VARCHAR(100000),
        PRIMARY KEY(movie_id)
    );
    """
)

C:\Users\fevc_\AppData\Local\Temp\ipykernel_12112\2042947556.py:2: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  engine.execute("""


In [6]:
# insert the values of the dataframes into the tables on 
tmdb_5000_movies_df.to_sql('tmdb_5000_movies', con = engine, if_exists = 'append')
tmdb_5000_credits_df.to_sql('tmdb_5000_credits', con = engine, if_exists = 'append')

# Drop the columns index
engine.execute(
    """
    ALTER TABLE tmdb_5000_movies
    DROP COLUMN index;
    
    ALTER TABLE tmdb_5000_movies
    DROP COLUMN title;
    
    ALTER TABLE tmdb_5000_credits
    DROP COLUMN index;
    """
)


In [7]:
# Join the tables
# Drop the table
engine.execute("""
DROP TABLE IF EXISTS tmdb_movies;
""")

# Create the table
engine.execute(
    """
    CREATE TABLE IF NOT EXISTs tmdb_movies AS(
        SELECT *
        FROM tmdb_5000_credits
        RIGHT JOIN tmdb_5000_movies
        ON tmdb_5000_credits.movie_id=tmdb_5000_movies.id
    );
    """
)

In [8]:
# Drop the columns that we do not need from tmdb_movies
engine.execute(
    """
    ALTER TABLE tmdb_movies DROP COLUMN budget;
    ALTER TABLE tmdb_movies DROP COLUMN homepage;
    ALTER TABLE tmdb_movies DROP COLUMN id;
    ALTER TABLE tmdb_movies DROP COLUMN original_language;
    ALTER TABLE tmdb_movies DROP COLUMN original_title;
    ALTER TABLE tmdb_movies DROP COLUMN popularity;
    ALTER TABLE tmdb_movies DROP COLUMN production_companies;
    ALTER TABLE tmdb_movies DROP COLUMN production_countries;
    ALTER TABLE tmdb_movies DROP COLUMN release_date;
    ALTER TABLE tmdb_movies DROP COLUMN revenue;
    ALTER TABLE tmdb_movies DROP COLUMN runtime;
    ALTER TABLE tmdb_movies DROP COLUMN spoken_languages;
    ALTER TABLE tmdb_movies DROP COLUMN status;
    ALTER TABLE tmdb_movies DROP COLUMN tagline;
    ALTER TABLE tmdb_movies DROP COLUMN vote_average;
    ALTER TABLE tmdb_movies DROP COLUMN vote_count;
    """
)

In [9]:
# Commit the DATABASE
# Create a database session
session = Session(engine)

# Your database operations go here

# Commit changes
session.commit()

# Close the session
session.close()


In [11]:
# creat a dataframe from tmdb_movies
tmdb_movies_df = pd.read_sql_table('tmdb_movies', con = engine)
tmdb_movies_df.to_csv('../data/interim/tmdb_movies.csv', index=False)
tmdb_movies_df

,movie_id,title,casting,crew,genres,keywords,overview
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",Following the death of District Attorney Harve...
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 5616, ""name"": ""united states\u2013mexi...",El Mariachi just wants to play his guitar and ...
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",[],A newlywed couple's honeymoon is upended by th...
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de...","[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...","[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de...",[],[],When ambitious New York attorney Sam is sent t...


### **Step 3:** Transform the data